<a href="https://colab.research.google.com/github/grese/scad-tot/blob/master/reaction_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import math, os
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from IPython.display import display, HTML
from sklearn import metrics
from sklearn.model_selection import train_test_split

TensorFlow 2.x selected.


## Authorize Google Drive

In [0]:
auth.authenticate_user()

## Setup

In [0]:
tensorboard_logs = './logs'
pd.options.display.float_format = '{:.2f}'.format

def print_html(string, tag='span', color=None, size=None):
    size = f'font-size:{size};' if size else ''
    color = f'color:{color};' if color else ''
    display(HTML(f'<{tag} style="{color}{size}">{string}</{tag}>'))

def print_heading(string, color=None):
    print_html(string, tag='h3', color=color)

def print_message(string, color=None):
    print_html(string, color=color)

def download_csv_from_gdrive(gdrive_id, output_file):
    # Authenticate google drive
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    # Download csv from GDrive...
    print_message(f'Downloading {gdrive_id} from GDrive. This may take a few minutes...')
    dataset = drive.CreateFile({'id': gdrive_id})
    dataset.GetContentFile(output_file)
    print_message(f'Downloaded {gdrive_id}.')

## Download Dataset

In [4]:
# GDrive ID of csv file (AllData.csv)
# https://drive.google.com/file/d/1vJfTw14BUDiWxtzt_9um41l5Afy0e3uP
gdrive_id='1vJfTw14BUDiWxtzt_9um41l5Afy0e3uP'
dataset_file='all_data.csv'

# load the dataset from gdrive if it doesn't exist in the runtime's filesystem.
if not os.path.exists(dataset_file):
    download_csv_from_gdrive(gdrive_id, dataset_file)

## Raw Dataset

In [5]:
# Read all_data.csv file from runtime's filesystem.
print_message('Reading data from filesystem. This may take a minute...')
raw_df = pd.read_csv(dataset_file)

# Display the raw/unsanitized dataset:
print_heading('Raw Dataset:')
display(raw_df)
print_heading('Raw Summary:')
display(raw_df.describe())

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,AccX,AccY,AccZ,Age,Alarm (0.0),Annotation,AutoBrake (0.0),AutoGear (0.0),AutoThrottle (0.0),AutoWheel (0.0),CurrentBrake (0.0),CurrentGear (0.0),CurrentThrottle (0.0),CurrentWheel (0.0),Distance3D,EventN (0.0),EventSource,EventW (0.0),ExportDate,FixationAOI,FixationDuration,FixationSeq,FixationStart,FixationX,FixationY,GSR CAL (kOhms) (GSR),GSR CAL (µSiemens) (GSR),GSR Quality (GSR),GSR RAW (no units) (GSR),Gaze3DX,Gaze3DY,Gaze3DZ,GazeAOI,GazeDirectionLeftX,GazeDirectionLeftY,GazeDirectionLeftZ,GazeDirectionRightX,GazeDirectionRightY,GazeDirectionRightZ,...,MediaTime,Mode (0.0),Name,PDTOn (0.0),PDTUser (0.0),Packet reception rate RAW (no units) (GSR),PassBy (0.0),PostMarker,PupilLeft,PupilRight,RangeN (0.0),RangeW (0.0),RightLaneDist (0.0),RightLaneType (0.0),SaccadeDuration,SaccadeSeq,SaccadeStart,SceneOutput,SceneParent,SceneType,SlideType,Speed (0.0),StimulusName,StudyName,System Timestamp CAL (mSecs) (GSR),TimeSignal,Timestamp,Timestamp CAL (mSecs) (GSR),Timestamp RAW (no units) (GSR),Trust (0.0),VSenseBatt CAL (mVolts) (GSR),VSenseBatt RAW (no units) (GSR),ValidityLeft,ValidityRight,Wide Range Accelerometer X CAL (m/(sec^2)) (GSR),Wide Range Accelerometer X RAW (no units) (GSR),Wide Range Accelerometer Y CAL (m/(sec^2)) (GSR),Wide Range Accelerometer Y RAW (no units) (GSR),Wide Range Accelerometer Z CAL (m/(sec^2)) (GSR),Wide Range Accelerometer Z RAW (no units) (GSR)
0,0,nan,nan,nan,27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,GSR GSR,nan,20191213,nan,nan,nan,nan,nan,nan,135.15,7.40,VALID,18406.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,018_M3,nan,nan,99.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,BlackInterslide,nan,Tobii Glasses 2 Scene,TakeOver_Final,1576095591013.68,nan,113,3853437.50,11906465.00,nan,3626.46,2490.00,nan,nan,6.15,-8752.00,5.24,10272.00,-1.40,2344.00
1,1,nan,nan,nan,27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,GSR GSR,nan,20191213,nan,nan,nan,nan,nan,nan,135.34,7.39,VALID,18404.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,018_M3,nan,nan,99.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,BlackInterslide,nan,Tobii Glasses 2 Scene,TakeOver_Final,1576095591021.50,nan,119,3853445.31,11906721.00,nan,3597.33,2470.00,nan,nan,6.02,-8368.00,5.01,10060.00,-1.41,2352.00
2,2,nan,nan,nan,27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,GSR GSR,nan,20191213,nan,nan,nan,nan,nan,nan,135.34,7.39,VALID,18404.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,018_M3,nan,nan,99.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,BlackInterslide,nan,Tobii Glasses 2 Scene,TakeOver_Final,1576095591029.31,nan,119,3853453.12,11906977.00,nan,3625.01,2489.00,nan,nan,5.46,-7612.00,4.56,9120.00,-1.65,2752.00
3,3,nan,nan,nan,27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,GSR GSR,nan,20191213,nan,nan,nan,nan,nan,nan,135.54,7.38,VALID,18402.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,018_M3,nan,nan,99.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,BlackInterslide,nan,Tobii Glasses 2 Scene,TakeOver_Final,1576095591037.12,nan,133,3853460.94,11907233.00,nan,3625.01,2489.00,nan,nan,5.10,-7836.00,4.69,8516.00,-1.51,2528.00
4,4,nan,nan,nan,27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,GSR GSR,nan,20191213,nan,nan,nan,nan,nan,nan,135.54,7.38,VALID,18402.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,018_M3,nan,nan,99.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,BlackInterslide,nan,Tobii Glasses 2 Scene,TakeOver_Final,1576095591044.93,nan,147,3853468.75,11907489.00,nan,3625.01,2489.00,nan,nan,7.56,-9012.00,5.39,12636.00,-2.07,3452.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9884279,9884279,0.39,-9.76,-0.21,29,346.00,nan,0.00

,Unnamed: 0,AccX,AccY,AccZ,Age,Alarm (0.0),Annotation,AutoBrake (0.0),AutoGear (0.0),AutoThrottle (0.0),AutoWheel (0.0),CurrentBrake (0.0),CurrentGear (0.0),CurrentThrottle (0.0),CurrentWheel (0.0),Distance3D,EventN (0.0),EventW (0.0),ExportDate,FixationAOI,FixationDuration,FixationSeq,FixationStart,FixationX,FixationY,GSR CAL (kOhms) (GSR),GSR CAL (µSiemens) (GSR),GSR RAW (no units) (GSR),Gaze3DX,Gaze3DY,Gaze3DZ,GazeAOI,GazeDirectionLeftX,GazeDirectionLeftY,GazeDirectionLeftZ,GazeDirectionRightX,GazeDirectionRightY,GazeDirectionRightZ,GazeEventType,GazeLeftx,...,ManualGear (0.0),ManualThrottle (0.0),ManualWheel (0.0),MarkerText,MediaTime,Mode (0.0),PDTOn (0.0),PDTUser (0.0),Packet reception rate RAW (no units) (GSR),PassBy (0.0),PostMarker,PupilLeft,PupilRight,RangeN (0.0),RangeW (0.0),RightLaneDist (0.0),RightLaneType (0.0),SaccadeDuration,SaccadeSeq,SaccadeStart,SceneOutput,SceneParent,SceneType,Speed (0.0),System Timestamp CAL (mSecs) (GSR),TimeSignal,Timestamp,Timestamp CAL (mSecs) (GSR),Timestamp RAW (no units) (GSR),Trust (0.0),VSenseBatt CAL (mVolts) (GSR),VSenseBatt RAW (no units) (GSR),ValidityLeft,ValidityRight,Wide Range Accelerometer X CAL (m/(sec^2)) (GSR),Wide Range Accelerometer X RAW (no units) (GSR),Wide Range Accelerometer Y CAL (m/(sec^2)) (GSR),Wide Range Accelerometer Y RAW (no units) (GSR),Wide Range Accelerometer Z CAL (m/(sec^2)) (GSR),Wide Range Accelerometer Z RAW (no units) (GSR)
count,9884284.00,7393997.00,7393997.00,7393997.00,9884284.00,9789281.00,0.00,9789285.00,9789284.00,9789285.00,9789285.00,9789286.00,9789285.00,9789293.00,9789286.00,7393997.00,9789278.00,9789278.00,9884284.00,0.00,5881390.00,5881390.00,5881390.00,5881390.00,5881390.00,9884279.00,9884279.00,9884279.00,7393997.00,7393997.00,7393997.00,0.00,7393997.00,7393997.00,7393997.00,7393997.00,7393997.00,7393997.00,0.00,9873006.00,...,9789282.00,9789283.00,9789282.00,0.00,9869517.00,9789282.00,9789269.00,9789268.00,9884279.00,9789270.00,0.00,9873006.00,9873006.00,9789276.00,9789278.00,9789272.00,9789272.00,0.00,0.00,0.00,0.00,0.00,0.00,9789278.00,9884279.00,9873006.00,9884284.00,9884279.00,9884279.00,9789281.00,9884279.00,9884279.00,7393997.00,7393997.00,9884279.00,9884279.00,9884279.00,9884279.00,9884279.00,9884279.00
mean,4942141.50,-0.08,-9.37,-0.17,23.72,19.74,nan,0.00,1.00,17.79,1.71,2.21,1.00,16.53,3.10,673.30,19.74,16.65,20191213.00,nan,653.27,924.73,504640.50,885.74,432.33,514.20,5.11,27893.48,10.89,63.28,655.78,nan,-0.02,0.07,0.85,0.05,0.08,0.82,nan,-16691.41,...,1.01,4.33,-1.65,nan,528015.38,0.66,0.75,0.01,99.04,0.02,nan,4.59,4.64,13.01,7.55,1.75,2.78,nan,nan,nan,nan,nan,nan,14.59,1575613506240.84,510137.02,528725.94,2706604.82,8432297.28,4.23,3940.93,2705.92,0.82,0.82,5.38,-2282.40,1.37,8998.12,-0.45,758.89
std,2853347.16,0.79,1.84,1.56,3.26,67.54,nan,0.00,0.00,2.48,124.14,62.20,0.00,10.43,36.50,1495.73,67.54,63.02,0.00,nan,1043.82,621.92,329332.28,306.65,213.61,1692.27,3.20,11509.23,233.87,212.67,1468.48,nan,0.14,0.20,0.32,0.13,0.19,0.35,nan,7190847.98,...,0.10,15.91,5.54,nan,330864.78,0.75,0.43,0.09,0.92,0.15,nan,0.75,0.73,40.08,26.23,0.42,0.63,nan,nan,nan,nan,nan,nan,2.87,238141576.48,326233.15,331270.44,1605711.10,4808673.11,1.52,143.57,98.57,1.85,1.85,4.52,5809.16,3.48,7554.09,5.45,9100.96
min,0.00,-19.93,-20.13,-13.05,20.00,0.00,nan,0.00,1.00,15.00,-3800.00,0.00,1.00,0.00,-3740.00,0.00,0.00,0.00,20191213.00,nan,100.00,1.00,4.00,0.00,0.00,-1748251.75,-0.02,17438.00,-27753.64,-10753.86,0.00,nan,-0.89,-0.99,-0.22,-0.85,-1.00,-0.50,nan,-2147483648.00,...,1.00,0.00,-90.00,nan,0.00,-1.00,0.00,0.00,88.00,0.00,nan,1.60,1.48,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,0.00,1575310311409.01,0.00,0.00,13195.31,17.00,1.00,3441.50,2363.00,0.00,0.00,-19.59,-32736.00,-19.59,-32736.00,-19.59,-32736.00
25%,2471070.75,-0.43,-9.86,-1.36,21.00,0.00,nan,0.00,1.00,15.00,-16.00,0.00,1.00,15.00,-8.00,328.78,0.00,0.00,20191213.00,nan,219.00,367.00,199919.00,848.00,309.00,129.84,2.18,18323.00,-13.49,0.00,316.16,nan,-0.07,0.00,0.93,0.00,0.00,0.92

## Sanitized Dataset

In [6]:
alarm_values = [0, 5, 16, 84, 103, 137, 178, 185, 191, 213, 254, 259, 284, 323, 332, 339]
column_names = ['GazeLeftx', 'GazeLefty', 'GazeRightx', 'GazeRighty', 'PupilLeft', 'PupilRight', 'ValidityLeft', 'ValidityRight', 'AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'Gaze3DX', 'Gaze3DY', 'Gaze3DZ', 'Distance3D', 'GazeDirectionLeftX', 'GazeDirectionLeftY', 'GazeDirectionLeftZ', 'GazeDirectionRightX', 'GazeDirectionRightY', 'GazeDirectionRightZ', 'GazeX', 'GazeY', 'InterpolatedGazeX', 'InterpolatedGazeY', 'FixationSeq', 'FixationX', 'FixationY', 'FixationStart', 'FixationDuration', 'Trust (0.0)', 'TimeSignal', 'Timestamp', 'Mode (0.0)', 'Alarm (0.0)']
dtypes = {}
for c in column_names: dtypes[c] = np.float64

# Read all_data.csv file from runtime's filesystem.
print_message('Reading data from filesystem. This may take a minute...')
df = pd.read_csv(dataset_file, usecols=column_names, dtype=dtypes)
print_message('Done.')

# Remove special chars from column names
renamed_cols = {'Alarm (0.0)': 'Alarm', 'Mode (0.0)': 'Mode', 'Trust (0.0)': 'Trust'}
df.rename(columns=renamed_cols, inplace=True)

# Fill NaN values with 0.0
df.fillna(0, inplace=True)

# drop rows with unknown alarm values
df.drop(df[~df.Alarm.isin(alarm_values)].index, inplace=True)

# Drop duplicate rows
df.drop_duplicates(inplace=True)

# Drop columns with no unique values in any rows
for col in df.columns:
       if len(df[col].unique()) == 1:
           df.drop(col, inplace=True, axis=1)

# Calculate take over time
df['TakeOverTime'] = df['Timestamp'] - df['TimeSignal']

In [7]:
display(df)
display(df.describe())

,AccX,AccY,AccZ,Alarm,Distance3D,FixationDuration,FixationSeq,FixationStart,FixationX,FixationY,Gaze3DX,Gaze3DY,Gaze3DZ,GazeDirectionLeftX,GazeDirectionLeftY,GazeDirectionLeftZ,GazeDirectionRightX,GazeDirectionRightY,GazeDirectionRightZ,GazeLeftx,GazeLefty,GazeRightx,GazeRighty,GazeX,GazeY,GyroX,GyroY,GyroZ,InterpolatedGazeX,InterpolatedGazeY,Mode,PupilLeft,PupilRight,TimeSignal,Timestamp,Trust,ValidityLeft,ValidityRight,TakeOverTime
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,113.00,0.00,0.00,0.00,113.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,119.00,0.00,0.00,0.00,119.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,133.00,0.00,0.00,0.00,133.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,147.00,0.00,0.00,0.00,147.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,154.00,0.00,0.00,0.00,154.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9881923,0.39,-9.91,0.20,0.00,374.37,360.00,311.00,163884.00,842.00,402.00,47.44,58.04,366.79,0.06,0.17,0.98,0.17,0.23,0.96,836.00,382.00,836.00,382.00,836.00,382.00,0.42,1.78,-1.25,836.00,382.00,1.00,3.92,4.08,163884.00,163884.00,5.00,0.00,0.00,0.00
9881924,0.39,-9.92,0.24,0.00,388.47,360.00,311.00,163884.00,842.00,402.00,48.38,55.83,381.38,0.07,0.16,0.98,0.17,0.22,0.96,839.00,395.00,839.00,395.00,839.00,395.00,0.28,1.91,-1.58,839.00,395.00,1.00,3.91,4.08,163904.00,163904.00,5.00,0.00,0.00,0.00
9881925,0.39,-9.92,0.24,0.00,388.47,360.00,311.00,163884.00,842.00,402.00,48.38,55.83,381.38,0.07,0.16,0.98,0.17,0.22,0.96,839.00,395.00,839.00,395.00,839.00,395.00,0.28,1.91,-1.58,839.00,395.00,1.00,3.91,4.08,163904.00,163906.00,5.00,0.00,0.00,2.00
9881926,0.39,-9.92,0.24,0.00,388.47,360.00,311.00,163884.00,842.00,402.00,48.38,55.83,381.38,0.07,0.16,0.98,0.17,0.22,0.96,839.00,395.00,839.00,395.00,839.00,395.00,0.28,1.91,-1.58,839.00,395.00,1.00,3.91,4.08,163904.00,163908.00,5.00,0.00,0.00,4.00


,AccX,AccY,AccZ,Alarm,Distance3D,FixationDuration,FixationSeq,FixationStart,FixationX,FixationY,Gaze3DX,Gaze3DY,Gaze3DZ,GazeDirectionLeftX,GazeDirectionLeftY,GazeDirectionLeftZ,GazeDirectionRightX,GazeDirectionRightY,GazeDirectionRightZ,GazeLeftx,GazeLefty,GazeRightx,GazeRighty,GazeX,GazeY,GyroX,GyroY,GyroZ,InterpolatedGazeX,InterpolatedGazeY,Mode,PupilLeft,PupilRight,TimeSignal,Timestamp,Trust,ValidityLeft,ValidityRight,TakeOverTime
count,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00,8003833.00
mean,-0.06,-7.24,-0.13,18.47,518.48,386.19,551.82,301381.58,533.73,258.99,8.39,48.51,504.93,-0.01,0.05,0.65,0.04,0.06,0.63,-17319.80,-9889.78,-17319.80,-9889.78,785.06,399.82,0.83,1.15,-0.75,784.36,399.33,0.65,4.58,4.63,511520.71,530054.03,4.19,0.63,0.63,18533.32
std,0.70,4.25,1.37,64.70,1347.41,862.74,660.97,355227.87,494.82,267.25,206.57,189.37,1322.28,0.12,0.17,0.45,0.12,0.17,0.46,7354169.14,5621515.85,7354169.14,5621515.85,421.72,261.83,8.27,15.48,5.37,422.05,261.81,0.75,0.77,0.75,326143.50,330757.28,1.56,1.66,1.66,92710.72
min,-19.93,-20.13,-13.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-27753.64,-10753.86,0.00,-0.89,-0.99,-0.22,-0.85,-1.00,-0.50,-2147483648.00,-2147483648.00,-2147483648.00,-2147483648.00,-1.00,-1.00,-194.46,-491.67,-349.19,-1.00,-1.00,-1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,-0.31,-9.84,-1.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.04,0.00,0.00,0.00,0.00,0.00,737.00,219.00,737.00,219.00,738.00,220.00,0.00,0.00,-1.55,736.00,219.00,1.00,3.94,4.04,220488.00,235123.00,5.00,0.00,0.00,2.00
50%,0.00,-9.69,0.00,0.00,429.69,160.00,249.00,131460.00,799.00,247.00,0.00,0.00,417.97,0.00,0.01,0.96,0.00,0.01,0.95,910.00,426.00,910.00,426.00,910.00,426.00,0.28,0.92,-0.44,910.00,426.00,1.00,4.74,4.91,488172.00,518731.00,5.00,0.00,0.00,11.00
75%,0.08,0.00,0.40,0.00,738.40,440.00,1035.00,588936.00,957.00,481.00,39.59,77.85,722.08,0.03,0.16,0.99,0.10,0.16,0.98,1016.00,572.00,1016.00,572.00,1016.00,572.00,1.82,2.26,0.00,1016.00,571.00,1.00,5.00,5.00,780125.00,809611.00,5.00,0.00,0.00,425.00
max,19.93,5.49,10.76,339.00,209837.60,15444.00,2613.00,1222804.00,1915.00,1078.00,37440.29,29179.03,209093.80,0.91,0.91,1.00,0.99,0.91,1.00,2060697600.00,976276800.00,2060697600.00,976276800.00,1919.00,1079.00,294.28,459.61,342.30,1919.00,1079.00,1.00,8.64,8.50,1223024.00,1223459.00,5.00,5.00,5.00,902432.00


In [0]:
# plotting entire dataset takes FOREVER! So, plotting random sample.
df_sample = df.sample(frac=0.01)

In [0]:
pd.plotting.scatter_matrix(df_sample, figsize=(80, 80))

In [0]:
sns.pairplot(df_sample, hue='Alarm')

In [0]:
# df_sample.plot(subplots=True, layout=(-1, 4), figsize=(80, 80), sharex=False)